In [6]:
import datetime
import glob
import os
import subprocess

import kfp

In [7]:
KUBEFLOW_URI = "https://52f46cb5ee9f784b-dot-us-central2.pipelines.googleusercontent.com/"

# Build Components
Since we are using custom Kubeflow components, we need to build the images for each of these custom components before they can be used in the pipeline.

In [20]:
os.chdir("..")

In [23]:
for component_folder in glob.glob("./components/**"):
    os.chdir(component_folder)
    subprocess.call(["gcloud", "builds", "submit", "--async"])
    os.chdir("../..")

# Execute Single model (SI)
We will first test out the Kubeflow Pipeline by executing a single (subset) XGBoost model.

## Compiling the pipeline 
The Kubeflow pipeline needs to be compiled from the Python definition

In [4]:
# !python baseball-pipeline-single.py

/opt/conda/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1028: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


## Submitting the pipeline to the Kubeflow cluster

In [6]:
client = kfp.Client(KUBEFLOW_URI)

In [29]:
response = client.upload_pipeline(
    "baseball-pipeline-single.py.zip", 
    pipeline_name="baseball-pipeline-single"
)

In [32]:
default_exp_id = client.experiments.list_experiment().experiments[0].id

In [33]:
client.run_pipeline(
    experiment_id=default_exp_id,
    job_name=f"xgb_single_{datetime.datetime.now().strftime('%m%d%H%M%S')}",
    pipeline_id=response.id
)

{'created_at': datetime.datetime(2020, 12, 6, 0, 38, 52, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': '97637878-8cd1-4575-bc37-0ca902b18670',
 'metrics': None,
 'name': 'xgb_single_1206003852',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': '01b4ebd2-56e2-46d5-b86f-5352e1399b52',
                   'pipeline_manifest': None,
                   'pipeline_name': 'baseball-pipeline-single',
                   'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"sequential-pipeline-","creationTimestamp":null,"labels":{"pipelines.kubeflow.org/kfp_sdk_version":"1.1.2"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.1.2","pipelines.kubeflow.org/pipeline_compilation_time":"2020-12-06T00:26:39.113245","pipelines.kubeflow.org/pipeline_spec":"{\\"description\\": '
                                        '\\"A pipeline w